## Part 4 : To predict the genre for test data across all the saved models

In [1]:
import findspark

In [2]:
findspark.init('/home/cse587/spark-2.4.0-bin-hadoop2.7')

In [3]:
import pyspark

In [4]:
pyspark

<module 'pyspark' from '/home/cse587/spark-2.4.0-bin-hadoop2.7/python/pyspark/__init__.py'>

In [5]:
from pyspark.sql import *

# Build the SparkSession
spark = SparkSession.builder \
   .appName("Assignment3p4testing") \
   .config("spark.some.config.option", "8gb") \
   .getOrCreate()
   


In [6]:
sc = spark.sparkContext
sc.setLogLevel("ERROR")

In [7]:
from pyspark.sql import SQLContext
sqlcontext=SQLContext(sc)

## Loading necessary libraries for this part

In [27]:
import numpy as np
import pandas as pd
import re
from pyspark.sql import SQLContext
sqlContext = SQLContext(sc)
import re
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.feature import HashingTF, IDF, Tokenizer
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.feature import RegexTokenizer, StopWordsRemover, CountVectorizer
from pyspark.sql.functions import udf, col, lower, regexp_replace
from pyspark.sql import functions as F
from pyspark.ml import Pipeline
from pyspark.ml.feature import OneHotEncoder, StringIndexer, VectorAssembler, VectorIndexer
from pyspark.mllib.linalg import SparseVector, DenseVector
from pyspark.ml.feature import StringIndexer, IndexToString
from pyspark.ml.classification import RandomForestClassificationModel
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.feature import OneHotEncoder, StringIndexer, VectorAssembler, VectorIndexer, IndexToString
from pyspark.ml.feature import Word2Vec

## Function to get predictions using Model from part1

In [22]:
def prediction_part1(training_df,testing_df,model):
    ## Defining Extra stopwords to maintain consistency with the trained model
    extra_stopwords =['http','https','amp','rt','t','c','the','despite','working','develop','whence', 'here', 'show', 'were', 'why', 
            'n’t', 'the', 'whereupon', 'not', 'more', 'how', 'eight', 'indeed', 'i', 'only', 'via', 'nine', 
            're', 'themselves', 'almost', 'to', 'already', 'front', 'least', 'becomes', 'thereby', 'doing', 
            'her', 'together', 'be', 'often', 'then', 'quite', 'less', 'many', 'they', 'ourselves', 'take',
            'its', 'yours', 'each', 'would', 'may', 'namely', 'do', 'whose', 'whether', 'side', 'both', 'what',
            'between', 'toward', 'our', 'whereby', "'m", 'formerly', 'myself', 'had', 'really', 'call', 'keep',
            "'re", 'hereupon', 'can', 'their', 'eleven', '’m', 'even', 'around', 'twenty', 'mostly', 'did', 
            'at', 'an', 'seems', 'serious', 'against', "n't", 'except', 'has', 'five', 'he', 'last', '‘ve', 
            'because', 'we', 'himself', 'yet', 'something', 'somehow', '‘m', 'towards', 'his', 'six',
            'anywhere', 'us', '‘d', 'thru', 'thus', 'which', 'everything', 'become', 'herein', 'one', 'in',
            'although', 'sometime', 'give', 'cannot', 'besides', 'across', 'noone', 'ever', 'that', 'over', 
            'among', 'during', 'however', 'when', 'sometimes', 'still', 'seemed', 'get', "'ve", 'him', 'with', 
            'part', 'beyond', 'everyone', 'same', 'this', 'latterly', 'no', 'regarding', 'elsewhere', 'others', 
            'moreover', 'else', 'back', 'alone', 'somewhere', 'are', 'will', 'beforehand', 'ten', 'very',
            'most', 'three', 'former', '’re', 'otherwise', 'several', 'also', 'whatever', 'am', 'becoming', 
            'beside', '’s', 'nothing', 'some', 'since', 'thence', 'anyway', 'out', 'up', 'well', 'it',
            'various', 'four', 'top', '‘s', 'than', 'under', 'might', 'could', 'by', 'too', 'and',
            'whom', '‘ll', 'say', 'therefore', "'s", 'other', 'throughout', 'became', 'your', 'put', 'per', 
            "'ll", 'fifteen', 'must', 'before', 'whenever', 'anyone', 'without', 'does', 'was', 'where', 
            'thereafter', "'d", 'another', 'yourselves', 'n‘t', 'see', 'go', 'wherever', 'just', 'seeming', 
            'hence', 'full', 'whereafter', 'bottom', 'whole', 'own', 'empty', 'due', 'behind', 'while', 'onto', 
            'wherein', 'off', 'again', 'a', 'two', 'above', 'therein', 'sixty', 'those', 'whereas', 'using', 
            'latter', 'used', 'my', 'herself', 'hers', 'or', 'neither', 'forty', 'thereupon', 'now', 'after',
            'yourself', 'whither', 'rather', 'once', 'from', 'until', 'anything', 'few', 'into', 'such', 'being',
            'make', 'mine', 'please', 'along', 'hundred', 'should', 'below', 'third', 'unless', 'upon', 'perhaps',
            'ours', 'but', 'never', 'whoever', 'fifty', 'any', 'all', 'nobody', 'there', 'have', 'anyhow', 'of', 
            'seem', 'down', 'is', 'every', '’ll', 'much', 'none', 'further', 'me', 'who', 'nevertheless', 'about',
            'everywhere', 'name', 'enough', '’d', 'next', 'meanwhile', 'though', 'through', 'on', 'first', 'been', 
            'hereby', 'if', 'move', 'so', 'either', 'amongst', 'for', 'twelve', 'nor', 'she', 'always', 'these', 
            'as', '’ve', 'amount', '‘re', 'someone', 'afterwards', 'you', 'nowhere', 'itself', 'done', 'hereafter',
            'within', 'made', 'ca', 'them']
    testing_df= testing_df.withColumn("plot",F.lower(F.col('plot')))
    regexTokenizer = RegexTokenizer(inputCol="plot", outputCol="words", pattern="\\W")

    remover = StopWordsRemover(inputCol="words", outputCol="filtered" ).setStopWords(extra_stopwords)

    countVectors = CountVectorizer(inputCol="filtered", outputCol="features",minDF=120)
    pipeline = Pipeline(stages=[regexTokenizer, remover, countVectors])
    
    ## fitting the pipeline
    data = pipeline.fit(training_df)
    
    rescaledData = data.transform(training_df)
    label_stringIdx = StringIndexer(inputCol = "genre", outputCol = "label")
    fitter = label_stringIdx.fit(rescaledData)
#     rescaledData = fitter.transform(rescaledData)
    labels = fitter.labels
    
    test_data = data.transform(testing_df)
    ## fitting the model
    test_pred = model.transform(test_data)
#     print("Prediction from model of part1")
#     test_pred.select('movie_id','prediction').show()

    ## converted the predicted labels to its corresponding genre
    converter = IndexToString(inputCol="prediction", outputCol="originalCategory", labels = labels)
    test_result = converter.transform(test_pred)
    
    return test_result

## Function to get predictions using Model from part 2

In [23]:
def prediction_part2(training_df,testing_df,model):
    
    tokenizer = Tokenizer(inputCol="plot", outputCol="words")
    hashingTF = HashingTF(inputCol="words", outputCol="rawFeatures", numFeatures=20)
    idf = IDF(inputCol="rawFeatures", outputCol="features")

    pipeline = Pipeline(stages=[tokenizer, hashingTF, idf])
    ## fitting the pipeline
    
    fitter = pipeline.fit(training_df)
    
    rescaledData = fitter.transform(training_df)
    label_stringIdx = StringIndexer(inputCol = "genre", outputCol = "label")
    lbl = label_stringIdx.fit(rescaledData)
#     rescaledData = fitter.transform(rescaledData)
    labels = lbl.labels
    
    
    test_data2 = fitter.transform(testing_df)
    ## fitting the model
    test_pred = model.transform(test_data2)

    ## converted the predicted labels to its corresponding genre
    converter = IndexToString(inputCol="prediction", outputCol="originalCategory", labels = labels)
    test_result = converter.transform(test_pred)

    return test_result


## Function to get predictions using Model from part 3

In [29]:
def prediction_part3(training_df,testing_df,model):
    
    regexTokenizer = RegexTokenizer(inputCol="plot", outputCol="words", pattern="\\W")
    word2vec = Word2Vec(vectorSize = 100, minCount = 5, inputCol = 'words', outputCol = 'features')

    pipeline = Pipeline(stages=[regexTokenizer, word2vec])
    ## fitting the pipeline
    fitter = pipeline.fit(training_df)
    
    rescaledData = fitter.transform(training_df)
    label_stringIdx = StringIndexer(inputCol = "genre", outputCol = "label")
    lbl = label_stringIdx.fit(rescaledData)
#     rescaledData = fitter.transform(rescaledData)
    labels = lbl.labels
    
    test_data = fitter.transform(testing_df)
    ## fitting the model
    test_pred = model.transform(test_data)

    ## converted the predicted labels to its corresponding genre
    converter = IndexToString(inputCol="prediction", outputCol="originalCategory", labels = labels)
    test_result = converter.transform(test_pred)

    return test_result
    

## Defining function to map and extract the final predictions 

In [31]:
def map_and_extract(result):
    mapping = pd.read_csv('/home/cse587/Project3/mapping.csv')
    colnames = ['label','name']
    mapping.columns =colnames
    mapping_df = spark.createDataFrame(mapping)
    def convert(lines):
        lines = lines[1:-1].split(', ')
        temp=[0]*20
        for i in lines:   
            for key,value in mapping.iterrows():
                if i == ('\''+ value['name'] +'\''):
                    temp[key] = 1
                    continue
        return " ".join(map(str,temp))

    converter = udf(convert)
    result = result.withColumn("predictions",converter('originalCategory'))
    final_data = result.select("movie_id","predictions")
    
    return final_data
    

## Loading test data from local using Pandas (Enter the test data path here)

In [13]:
test_data_path = "/home/cse587/Project3/test.csv"
test_data = pd.read_csv(test_data_path,sep=",", header = 0)

## Creating testing dataframe in pyspark

In [14]:
testing_df = spark.createDataFrame(test_data)


## Loading and saving training dataset

In [15]:
## Specify training data path
train_data = pd.read_csv("/home/cse587/Project3/train.csv",sep=",", header = 0)
training_df = spark.createDataFrame(train_data)

## Predicting the result for each part

In [32]:
## paths of saved models
path1 = "/home/cse587/Project3/randomforest_for_part1"
path2 = "/home/cse587/Project3/randomforest_for_part2"
path3 = "/home/cse587/Project3/randomforest_for_part3"

## loading the saved models from all parts
from pyspark.ml.classification import RandomForestClassificationModel
saved_model1 = RandomForestClassificationModel.load(path1)
saved_model2 = RandomForestClassificationModel.load(path2)
saved_model3 = RandomForestClassificationModel.load(path3)

## getting the predictions from all models for a given test data 
part1_result = prediction_part1(training_df,testing_df,saved_model1)
part2_result = prediction_part2(training_df,testing_df,saved_model2)
part3_result = prediction_part3(training_df,testing_df,saved_model3)

## mapping the prediction to binary string using mapping.csv
part1_final = map_and_extract(part1_result)
part2_final = map_and_extract(part2_result)
part3_final = map_and_extract(part3_result)

## saving the results in .csv format , specify file name if required

part1_final.write.csv('part1_test_result_rf_video.csv', header = True)
part2_final.write.csv('part2_test_result_rf_video.csv', header = True)
part3_final.write.csv('part3_test_result_rf_video.csv', header = True)
